In [2]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
# resdir = r'C:\Users\jiahuic\Dropbox (University of Michigan)\Ford_CC\\'
resdir = r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\\'
# resdir = r'G:\Dropbox (University of Michigan)\Ford_CC\\'
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [3]:
gaspricedf = pd.read_csv(resdir+r'\\CostEmissionAssessment_revision\U.S._Regular_All_Formulations_Retail_Gasoline_Prices.csv',skiprows=5)
gasprice = np.mean(gaspricedf['Prices'][0:36])

In [4]:
# qual lifetime
# EV aggregate
resdir = r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\\'
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# Read base data
ctdf_withcarb = pd.read_csv(resdir+r'\\2020_Gaz_counties_national_0728_tempreg.csv')
uqlst = np.unique(ctdf_withcarb['tempreg'])

# Pre-allocate columns to avoid fragmentation
columns_to_add = []
for year in ['24','30','40']:
    for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons']:
        columns_to_add.extend([
            f'{year}{scen}costs', f'{year}{scen}emissions'
        ])

# Initialize all columns with zeros
for col in columns_to_add:
    ctdf_withcarb[col] = 0

# Format FIPS codes
ctdf_withcarb['fips'] = ctdf_withcarb.apply(lambda row: '0'+str(row['fips']) if len(str(row['fips']))==4 else str(row['fips']), axis=1)

# Process data for each year
for year in ['24','30','40']:
    # Read all scenario data files
    data_files = {
        'UC': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_0_0_0_0_3.csv',
        'CC': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_1_0_0_0_1.csv',
        'V2H': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_1.csv',
        'V2Hnoelec': resdir+'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_noelec_1.csv',
        'V2H_cons': resdir+'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_cons.csv'
    }
    
    # Process each scenario
    for scen, file_std in data_files.items():
        df_std = pd.read_csv(file_std)
        
        # Check for NaN values
        if df_std['Cost'].isna().any() or df_std['Emissions'].isna().any():
            print(f"Warning: NaN values found in {scen} data")
            
        # Assign values
        # Map values based on county_num matching tempreg
        for idx, row in df_std.iterrows():
            mask = ctdf_withcarb['tempreg'] == row['county_num']
            ctdf_withcarb.loc[mask, f'{year}{scen}costs'] = row['Cost']
            ctdf_withcarb.loc[mask, f'{year}{scen}emissions'] = row['Emissions']

# Calculate lifetime costs and emissions
for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons']:
    ctdf_withcarb[f'lca{scen}costs'] = 5 * (ctdf_withcarb[f'24{scen}costs'] + 
                                   ctdf_withcarb[f'30{scen}costs'] + 
                                   ctdf_withcarb[f'40{scen}costs'])
    ctdf_withcarb[f'lca{scen}emissions'] = 5 * (ctdf_withcarb[f'24{scen}emissions'] + 
                                       ctdf_withcarb[f'30{scen}emissions'] + 
                                       ctdf_withcarb[f'40{scen}emissions'])

# Calculate differences between scenarios (only for EV scenarios)
ev_scenarios = ['UC','CC','V2Hnoelec','V2H','V2H_cons']
for item in ['emissions','costs']:
    for scen0 in ev_scenarios:
        for scen1 in ev_scenarios:
            if scen0 != scen1:
                ctdf_withcarb[f'lca{scen0}{scen1}{item}'] = (
                    ctdf_withcarb[f'lca{scen0}{item}'] - 
                    ctdf_withcarb[f'lca{scen1}{item}']
                )

# Add duplicate row and update index
ctdf_withcarb = pd.concat([ctdf_withcarb, ctdf_withcarb.iloc[2330:2331]])
ctdf_withcarb['index'] = np.arange(ctdf_withcarb.shape[0])
ctdf_withcarb.set_index('index', inplace=True)
ctdf_withcarb.loc[ctdf_withcarb.index[-1], 'fips'] = '46113'

In [8]:
for scen in ['CCUC','V2HnoelecUC','V2HUC']:
    values = ctdf_withcarb[f'lca{scen}costs'].dropna().sort_values().values
    n = len(values)
    # Find the closest observation to the 5th and 95th percentiles
    p5_idx = int(round(0.05 * (n - 1)))
    p95_idx = int(round(0.95 * (n - 1)))
    mean_val = np.mean(values)
    print("5th:", values[p5_idx])
    print("Meanh:", mean_val)
    print("95th:", values[p95_idx])

5th: -2039.9840785347196
Meanh: -1752.070659771265
95th: -1356.1036406286703
5th: -5989.2033782956505
Meanh: -3897.988403343967
95th: -2541.5789796467498
5th: -5963.252204998398
Meanh: -4326.065596685214
95th: -3122.7828095237037


In [9]:
for scen in ['CCUC','V2HnoelecUC','V2HUC']:
    values = ctdf_withcarb[f'lca{scen}emissions'].dropna().sort_values().values
    n = len(values)
    # Find the closest observation to the 5th and 95th percentiles
    p5_idx = int(round(0.05 * (n - 1)))
    p95_idx = int(round(0.95 * (n - 1)))
    mean_val = np.mean(values)
    print("5th:", values[p5_idx]/1000)
    print("Meanh:", mean_val/1000)
    print("95th:", values[p95_idx]/1000)

5th: -25.117012774003875
Meanh: -21.808331384464946
95th: -18.65070509128916
5th: -83.42781937110504
Meanh: -62.00900982965741
95th: -39.077033696692396
5th: -109.46296725977618
Meanh: -86.33001523254039
95th: -59.98520719619557


In [10]:
# qual lifetime
# EV aggregate
resdir = r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\\'
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# Read base data
ctdf_nocarb = pd.read_csv(resdir+r'\\2020_Gaz_counties_national_0728_tempreg.csv')
uqlst = np.unique(ctdf_nocarb['tempreg'])

# Pre-allocate columns to avoid fragmentation
columns_to_add = []
for year in ['24','30','40']:
    for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons']:
        columns_to_add.extend([
            f'{year}{scen}costs', f'{year}{scen}emissions'
        ])

# Initialize all columns with zeros
for col in columns_to_add:
    ctdf_nocarb[col] = 0

# Format FIPS codes
ctdf_nocarb['fips'] = ctdf_nocarb.apply(lambda row: '0'+str(row['fips']) if len(str(row['fips']))==4 else str(row['fips']), axis=1)

# Process data for each year
for year in ['24','30','40']:
    # Read all scenario data files
    data_files = {
        'UC': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_0_0_0_0_3.csv',
        'CC': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_1_0_0_0_nocarb.csv',
        'V2H': resdir+r'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_nocarb.csv',
        'V2Hnoelec': resdir+'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_noelec_nocarb.csv',
        'V2H_cons': resdir+'\\CostEmissionAssessment\\res_trb\\20'+year+'_2_0_0_0_cons.csv'
    }
    
    # Process each scenario
    for scen, file_std in data_files.items():
        df_std = pd.read_csv(file_std)
        
        # Check for NaN values
        if df_std['Cost'].isna().any() or df_std['Emissions'].isna().any():
            print(f"Warning: NaN values found in {scen} data")
            
        # Assign values
        # Map values based on county_num matching tempreg
        for idx, row in df_std.iterrows():
            mask = ctdf_nocarb['tempreg'] == row['county_num']
            ctdf_nocarb.loc[mask, f'{year}{scen}costs'] = row['Cost']
            ctdf_nocarb.loc[mask, f'{year}{scen}emissions'] = row['Emissions']

# Calculate lifetime costs and emissions
for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons']:
    ctdf_nocarb[f'lca{scen}costs'] = 5 * (ctdf_nocarb[f'24{scen}costs'] + 
                                   ctdf_nocarb[f'30{scen}costs'] + 
                                   ctdf_nocarb[f'40{scen}costs'])
    ctdf_nocarb[f'lca{scen}emissions'] = 5 * (ctdf_nocarb[f'24{scen}emissions'] + 
                                       ctdf_nocarb[f'30{scen}emissions'] + 
                                       ctdf_nocarb[f'40{scen}emissions'])

# Calculate differences between scenarios (only for EV scenarios)
ev_scenarios = ['UC','CC','V2Hnoelec','V2H','V2H_cons']
for item in ['emissions','costs']:
    for scen0 in ev_scenarios:
        for scen1 in ev_scenarios:
            if scen0 != scen1:
                ctdf_nocarb[f'lca{scen0}{scen1}{item}'] = (
                    ctdf_nocarb[f'lca{scen0}{item}'] - 
                    ctdf_nocarb[f'lca{scen1}{item}']
                )

# Add duplicate row and update index
ctdf_nocarb = pd.concat([ctdf_nocarb, ctdf_nocarb.iloc[2330:2331]])
ctdf_nocarb['index'] = np.arange(ctdf_nocarb.shape[0])
ctdf_nocarb.set_index('index', inplace=True)
ctdf_nocarb.loc[ctdf_nocarb.index[-1], 'fips'] = '46113'

In [13]:
for scen in ['CCUC','V2HnoelecUC','V2HUC']:
    values = ctdf_nocarb[f'lca{scen}costs'].dropna().sort_values().values
    n = len(values)
    # Find the closest observation to the 5th and 95th percentiles
    p5_idx = int(round(0.05 * (n - 1)))
    p95_idx = int(round(0.95 * (n - 1)))
    mean_val = np.mean(values)
    print("5th:", round(values[p5_idx]/1000, 2))
    print("Meanh:", round(mean_val/1000, 2))
    print("95th:", round(values[p95_idx]/1000, 2))

5th: -2.11
Meanh: -1.68
95th: -1.17
5th: -6.23
Meanh: -4.24
95th: -2.75
5th: -6.11
Meanh: -4.69
95th: -3.45


In [16]:
len([i for i in range(20)][10:15])

5

In [20]:
import os
oldfiles = pd.read_csv(r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\old_files_report.csv')
oldfiles['directory'].apply(lambda x: os.path.dirname(x)).unique()

array(['C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h\\Results_80_Y82_2024_nocarb',
       'C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h\\Results_80_Y82_2030_nocarb',
       'C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h\\Results_80_Y82_2040_nocarb',
       'C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h_no\\Results_2024_nocarb',
       'C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h_no\\Results_2030_nocarb',
       'C:\\Users\\jiahuic\\University of Michigan Dropbox\\Jiahui Chen\\Ford_CC\\Result_v2h_no\\Results_2040_nocarb'],
      dtype=object)

In [1]:
import os
import pandas as pd
import numpy as np

# Set the directory path
base_dir = r'C:\Users\jiahuic\University of Michigan Dropbox\Jiahui Chen\Ford_CC\Result_v2h'

# Function to check if slowCR equals vSOC difference when slowCR isn't 0
def check_slowcr_vsoc(file_path):
    try:
        df = pd.read_csv(file_path)
        if 'slowCR' not in df.columns or 'vSOC' not in df.columns:
            return None
        
        # Calculate vSOC differences
        vsoc_diff = df['vSOC'].diff().fillna(0)
        
        # Only compare where slowCR is not 0
        mask = df['slowCR'] != 0
        if not any(mask):
            return None
            
        # Compare slowCR with vSOC differences only where slowCR isn't 0
        is_equal = np.isclose(df.loc[mask, 'slowCR'], vsoc_diff[mask], rtol=1e-5, atol=1e-5)
        
        if not all(is_equal):
            return {
                'file': file_path,
                'mismatch_indices': np.where(~is_equal)[0],
                'slowCR_values': df.loc[mask, 'slowCR'][~is_equal].values,
                'vsoc_diff_values': vsoc_diff[mask][~is_equal].values,
                'hour_indices': df.index[mask][~is_equal].values
            }
        return None
    except Exception as e:
        return {'file': file_path, 'error': str(e)}

# Walk through all directories and check CSV files
mismatches = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            result = check_slowcr_vsoc(file_path)
            if result:
                mismatches.append(result)

# Print results
if mismatches:
    print("Found mismatches between slowCR and vSOC differences (only where slowCR ≠ 0):")
    for mismatch in mismatches:
        if 'error' in mismatch:
            print(f"\nError in file {mismatch['file']}:")
            print(f"Error message: {mismatch['error']}")
        else:
            print(f"\nMismatches in file {mismatch['file']}:")
            print("Hour indices with mismatches:", mismatch['hour_indices'])
            print("slowCR values:", mismatch['slowCR_values'])
            print("vSOC diff values:", mismatch['vsoc_diff_values'])
            print("Absolute differences:", np.abs(mismatch['slowCR_values'] - mismatch['vsoc_diff_values']))
else:
    print("No mismatches found between slowCR and vSOC differences in any files (where slowCR ≠ 0).")



KeyboardInterrupt: 